In [ ]:
!pip install sagemaker -U

In [ ]:
import tensorflow as tf

# データロード
(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_valid = x_valid/255

# モデル定義
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding="same",activation="relu",input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10,activation="softmax"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),metrics=['accuracy'],loss="sparse_categorical_crossentropy")

# トレーニング
model.fit(x_train,y_train,batch_size=4,epochs=2,validation_data=(x_valid,y_valid))

# モデルの保存
model.save('./1')

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/4-1-1/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/4-1-2/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import tensorflow as tf
model = tf.keras.models.load_model('./1')
model.summary()

In [ ]:
!mkdir -p ./train
!mkdir -p ./valid

import tensorflow as tf
import numpy as np

(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_valid = x_valid/255

np.save('./train/x_train.npy',x_train)
np.save('./train/y_train.npy',y_train)
np.save('./valid/x_valid.npy',x_valid)
np.save('./valid/y_valid.npy',y_valid)
train_s3_uri = sagemaker.session.Session().upload_data(path='./train/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/4-1/train')
valid_s3_uri = sagemaker.session.Session().upload_data(path='./valid/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/4-1/valid')

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/4-1-3/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit({
    'train':train_s3_uri,
    'valid':valid_s3_uri,
})

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!rm model.tar.gz
!rm -rf ./1
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import tensorflow as tf
model = tf.keras.models.load_model('./1')
model.summary()

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/4-1-4/train.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'filters': 8,
        'epochs': 1,
        'batch_size': 16,
        'learning_rate': 0.001
    }
)
estimator.fit({
    'train':train_s3_uri,
    'valid':valid_s3_uri,
})

In [ ]:
# モデルの URI を取得
model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
# モデルをローカルにコピーして解凍
!rm model.tar.gz
!rm -rf ./1
!aws s3 cp {model_uri} ./
!tar zxvf model.tar.gz
# モデルの読み込み
import tensorflow as tf
model = tf.keras.models.load_model('./1')
model.summary()